# defind computional node class

In [88]:
class comp_node:
    def __init__(self, val, childern = [],op = "assign"):
        self.val = val
        # THE NODE i start backword is the parent and i go throw child
        self.childern = childern
        self.op = op
        self.grad = 0
        self.backward_prop=lambda: None
    
    def __to_comp_node(self, obj):
        # check if the other not object from comp_node class---> 
        # creat instant from comp_node for the value passed (other) 
        if not isinstance(obj, comp_node):  
            obj = comp_node(val = obj)
            return obj
        else:
            return obj

    def __repr__(self):
        return f"op:{self.op} | val: {self.val:.4f} | childern: {len(self.childern)} | grad: {self.grad}"


    def __sub__(self, other):
        # if not isinstance(other, comp_node):  
        #     other = comp_node(val = other)
        other = self.__to_comp_node(other)
        # creat instant to put sub in it, and this node is a parent for childern self, other 
        out = comp_node(val=self.val - other.val,childern =[self, other], op="sub" ) 
        def _backward_prop():
            self.grad += out.grad* (1)
            other.grad += out.grad *(-1)
        out.backward_prop=_backward_prop
        return out

        
    def __rsub__ (self, other): # its a rewrite of sub function 
        # right subtraction not left
        other =self. __to_comp_node(other)
      
        return other-self #i write the operation direcit as i defind the sub operation above 
    
    def __add__(self, other):
        other = self.__to_comp_node(other)
        out = comp_node(val=self.val + other.val,childern =[self, other], op="add" ) 
        def _backward_prop():
            self.grad += out.grad* (1)
            other.grad += out.grad *(1)
        out.backward_prop=_backward_prop
        return out
    
    def __radd__ (self, other): # its a rewrite of add function 
        other =self. __to_comp_node(other)
        return other + self #i write the operation direcit as i defind the add operation above 
    
    def __mul__(self, other):
        other = self.__to_comp_node(other)
        add = comp_node(val=self.val * other.val,childern =[self, other], op="mul" ) 
        return add
    
    def __rmul__ (self, other): 
        other =self. __to_comp_node(other)
      
        return other * self  

    def __pow__(self, exponent):
        if not isinstance(exponent,(int,float)):
            raise ValueError("unsupport type ")
        out= comp_node(val=self.val**exponent,childern=[self],op = f"power{exponent}") ##M
        def _backward_prop():
            self.grad += out.grad*(exponent*self.val**(exponent-1))  ### self.val===M
        out.backward_prop=_backward_prop
        return out

    def __eq__(self, other):
        return self.val==other.val
    


In [89]:
assert comp_node(5).val==5   ### assert is a way to check if my code work or no
#assert comp_node(7).val==5   ## it not work ---> give me an error

assert (comp_node(5) - comp_node(2)).val ==3 
assert (comp_node(5) - 2).val ==3
assert (3 - comp_node(val=5)).val == -2


assert (comp_node(5) + comp_node(2)).val == 7
assert (comp_node(5) + 2).val == 7
assert (3 + comp_node(val=5)).val == 8

assert (comp_node(5) * comp_node(2)).val == 10
assert (comp_node(5) * 2).val == 10
assert (3 * comp_node(val=5)).val == 15

assert (comp_node(val=5)**2).val==25
assert (comp_node(val=5)**2)== comp_node(val=25)

In [90]:
print(comp_node(val=5))

op:assign | val: 5.0000 | childern: 0 | grad: 0


# loss graph

In [91]:
from random import Random
seed= 5
data_gen=Random(seed)
data_x= [data_gen.uniform(a=0,b=1.) for _ in range (1000)]
data_y=[data_gen.uniform(a=0,b=1.)for _ in range (1000)]
xp,yp = comp_node(val=0.3),comp_node(val=0.3)



In [92]:
def loss_graph(xp,yp,data_x,data_y):
    Ix,Iy = xp - data_x, yp- data_y
    gx,gy = Ix**2, Iy**2
    M = gx + gy
    l = M**0.5
    return l,[l,M,gx,gy,Ix,Iy,xp,yp]


In [93]:
loss,rev_topo_order = loss_graph(xp,yp,data_x[0],data_y[0])
rev_topo_order[0].grad=1
for i, node in enumerate(rev_topo_order):
    node.backward_prop()
    print(i,node)


0 op:power0.5 | val: 0.6563 | childern: 1 | grad: 1
1 op:add | val: 0.4307 | childern: 2 | grad: 0.7618990103238323
2 op:power2 | val: 0.1043 | childern: 1 | grad: 0.7618990103238323
3 op:power2 | val: 0.3264 | childern: 1 | grad: 0.7618990103238323
4 op:sub | val: -0.3229 | childern: 2 | grad: -0.49203696353670395
5 op:sub | val: -0.5713 | childern: 2 | grad: -0.8705743084387342
6 op:assign | val: 0.3000 | childern: 0 | grad: -0.49203696353670395
7 op:assign | val: 0.3000 | childern: 0 | grad: -0.8705743084387342
